# Processing CSV files

In [1]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import data_processing as dp

pd.options.display.float_format = '{:.4f}'.format

IMU_list = ['BMI270', 'BMI055', 'ICM42688', 'MPU6000']

datasets = ['ATT', 'IMU']

# Adding fast sensor readings 
# datasets += ['RGY', 'RAC']

df_dict = dp.load_suspend_data(datasets,imu=IMU_list[3])

df_transition = dp.compute_transition_df(source_df=df_dict['GT'], param='Ground_Truth')

print("[**] transition dataframe")
print(df_transition)


[**] transition dataframe
       timestamp  Ground_Truth  transition
3440     14.9908           100    100.0000
5940     17.6972             0   -100.0000
9549     25.2972           100    100.0000
12049    28.0032             0   -100.0000
15775    35.6031           100    100.0000
18275    38.3090             0   -100.0000
21834    45.9076           100    100.0000
24334    48.6140             0   -100.0000
27817    56.2132           100    100.0000
30317    58.9191             0   -100.0000
34042    66.5194           100    100.0000


## Enriching Datasets

In [2]:
df=df_dict['GT']
param='Ground_Truth'
stats_params = {  
            'IMU': 
                ['GyrX', 'GyrY', 'GyrZ', 'AccX', 'AccY', 'AccZ'],
            # 'RAC':
            #     ['ACCx'],#'ACCy','ACCz'],
            # 'RGY':
            #     ['GYROx'],#'GYROy','GYROz'],
            'ATT':
                ['Roll', 'Pitch', 'Yaw']
        }


# Add attack column into based on timestamps from the transition dataframe
for k in stats_params.keys():
    attack0 = df[param].iloc[0]
    attack = list()
    this_df = df_dict[k]
    tr0 = 0
    for tr in df_transition['timestamp'].values:
        data_size = len(this_df[(this_df['timestamp']>tr0) & (this_df['timestamp']<=tr)])
        # print('adding {} data points to {} points'.format(data_size, len(attack)))
        attack += [attack0]*data_size
        attack0 = df_transition['Ground_Truth'][df_transition['timestamp']==tr].values[0]
        tr0 = tr
    # Compute last segment
    data_size = len(this_df[this_df['timestamp']>tr])
    # print('adding {} data points to {} points'.format(data_size, len(attack)))
    attack += [attack0]*data_size
    this_df['attack'] = attack
    
    # Add 'update' column with the time diference between data points
    this_df['update'] = this_df['timestamp'].diff()

    df_dict[k] = this_df

update_dict = dict()
for ds in datasets[:-1]:
    df_update = pd.DataFrame()
    df_update['timestamp'] = df_dict[ds]['timestamp']
    df_update['update'] = df_dict[ds]['timestamp'].diff()
    update_dict[ds] = df_update
    

# Statistical Results

In [3]:

stat_dict = dict()
param_set = list()

for k,values in stats_params.items():
    for v in values+['update']:
        this_df = df_dict[k]
        # tr0 = 0
        # compute mean and std deviation for all variables in 'stats_params'
        atk_set = this_df['attack'].unique()
        for atk in atk_set:
            if 'update' in v:
                filtered_list = 1/this_df[v][this_df['attack']==atk]
            else:
                filtered_list = this_df[v][this_df['attack']==atk]

            mean_value = filtered_list.mean()
            std_value = filtered_list.std()

            # fill up the stat_dict dictionary by attack
            name_code = '{}.{}'.format(k, v)
            param_set.append(name_code)
            if atk in stat_dict:
                    stat_dict[atk][name_code] = {'mean': mean_value, 'std':std_value}
            else:
                stat_dict[atk] = {name_code:{'mean': mean_value, 'std':std_value}}

# convert stats into a dataframe
df_columns = ['Freq']
for i in list(set(param_set)):
    df_columns+= [i+'.mean', i+'.std']
stat_df = pd.DataFrame(columns=df_columns)
stat_df['Freq'] = atk_set
for freq in stat_dict:
    for param in stat_dict[freq]:
        stat_df.loc[stat_df['Freq']==freq,param+'.mean'] = stat_dict[freq][param]['mean']
        stat_df.loc[stat_df['Freq']==freq,param+'.std'] = stat_dict[freq][param]['std']

pd.options.display.float_format = '{:.4f}'.format
print(stat_df.loc[stat_df['Freq']!=0])
print(stat_df.to_latex())

   Freq IMU.GyrZ.mean IMU.GyrZ.std ATT.Roll.mean ATT.Roll.std IMU.update.mean  \
1   100        0.1211       0.0209       -3.6657       3.2209        118.2109   

  IMU.update.std ATT.Pitch.mean ATT.Pitch.std ATT.Yaw.mean  ... IMU.AccX.mean  \
1        20.6739        39.1515        0.4573      18.4578  ...        0.2477   

  IMU.AccX.std IMU.GyrX.mean IMU.GyrX.std ATT.update.mean ATT.update.std  \
1       1.0581        0.0254       0.0042        120.8099        17.8207   

  IMU.GyrY.mean IMU.GyrY.std IMU.AccZ.mean IMU.AccZ.std  
1       -0.0346       0.0058       -0.3031       1.2947  

[1 rows x 23 columns]
\begin{tabular}{lrllllllllllllllllllllll}
\toprule
 & Freq & IMU.GyrZ.mean & IMU.GyrZ.std & ATT.Roll.mean & ATT.Roll.std & IMU.update.mean & IMU.update.std & ATT.Pitch.mean & ATT.Pitch.std & ATT.Yaw.mean & ATT.Yaw.std & IMU.AccY.mean & IMU.AccY.std & IMU.AccX.mean & IMU.AccX.std & IMU.GyrX.mean & IMU.GyrX.std & ATT.update.mean & ATT.update.std & IMU.GyrY.mean & IMU.GyrY.std & IMU

## Plotting Suspend attack results

In [4]:
import plot_formatting as pfg

fig = make_subplots(rows=4, cols=1,
                    shared_xaxes=True,
                    vertical_spacing = 0.02,
                    row_heights=[.25]*2 + [.25]*2)

colorset = px.colors.qualitative.Bold

# Plot data source
plot_params = {  
            'IMU': 
                ['GyrX', 'AccX'],
            # 'RAC':
            #     ['ACCx', 'update'],
            # 'RGY':
            #     ['GYROx', 'update'],
            'ATT':
                ['Roll', 'Pitch', 'Yaw', 'update'],
        }


plot_labels = {
            #     'RAC':'Accel', 
            #    'RGY': 'Gyro',
               'ATT': 'EKF Estimation'}

x_range = [55.2, 58]

color_ix = 0
for k,values in plot_params.items():
    for v in values:
        row_num  = 1
        this_df = df_dict[k]
        this_df = this_df.loc[(this_df['timestamp']>x_range[0]) & (this_df['timestamp']<x_range[1])]
        if 'G' in v:
            row_num = 2
        tr0 = 0
        # Plotting attitude
        if 'ATT' in k and not ('update' in v):
            fig.add_trace(go.Scatter(x=this_df['timestamp'], y=this_df[v],
                    name= v,
                    mode='lines',
                    legendgroup=k,
                    line=dict(color=colorset[color_ix]),
                    showlegend=True,
                    legendrank=3,
                    legendgrouptitle_text='Attitude'),
                row=3,col=1)
        # PLotting update frequencies
        elif 'update' in v:
            fig.add_trace(go.Scatter(x=this_df['timestamp'], y=1/this_df[v],
                    name= plot_labels[k],
                    mode='lines',
                    line=dict(color=colorset[color_ix]),
                    showlegend=True,
                    legendrank=5,
                    legendgroup='freq',
                    legendgrouptitle_text='Frequency'),
                row=4,col=1)
        else:
            # Plotting Accelerometer and Gyro
            fig.add_trace(go.Scatter(x=this_df['timestamp'], y=this_df[v],
                    name= k+str('.')+v,
                    mode='lines',
                    legendgroup=v,
                    line=dict(color=colorset[color_ix]),
                    showlegend=False),
                row=row_num,col=1)

        color_ix += 1


# Plot attacks
df = df_dict['GT']
df = df.loc[(df['timestamp']>x_range[0]) & (df['timestamp']<x_range[1])]


# inlcude attack transitions as vertical lines in the figure 
for v in df_transition['timestamp'][(df_transition['timestamp']>x_range[0]) & (df_transition['timestamp']<x_range[1])].values:
    fig.add_vline(x=v)

# beautify plots
fig = pfg.beautify_plot(fig)

subplot_titles=['value', 'data rate', 'freq'],

#  Formating x-axis
layout = go.Layout( autosize=True, margin={'l': 0, 'r': 0, 't': 20, 'b': 0})
fig['layout'].update(layout)
fig['layout']['xaxis'].update(range=x_range)
fig['layout']['xaxis4'].update(title='Time [s]', mirror=True)

# Formating y-axis
fig['layout']['yaxis1'].update(title='Accel X [m/s/s]')
fig['layout']['yaxis2'].update(title='Gyro X [rad/s]', tickvals = [-0.01, 0.00, 0.01, 0.02, 0.03])
fig['layout']['yaxis4'].update(title='Frequency [Hz]')
fig['layout']['yaxis3'].update(title='Attitude [°]')

fig['layout']['legend'].update(orientation='h',
                                x=0.6,
                                y=1,
                                   )

fig.show()


In [5]:
fig.write_image('figs/MPU6000Suspend.pdf')

## Plotting Frequency attack results

In [6]:
import data_processing as dp

IMU_list = ['BMI270', 'BMI055', 'ICM42688', 'MPU6000']

datasets = ['ATT', 'IMU']
# Adding fast sensor readings 
# datasets += ['RGY', 'RAC']

df_dict = dp.load_frequency_data(datasets,imu=IMU_list[3])

df_transition = dp.compute_transition_df(source_df=df_dict['AC'], param='Ground_Truth')
print("[**] transition dataframe")
print(df_transition)

[**] transition dataframe
       timestamp  Ground_Truth  transition
9718    164.6964            98    -90.0000
19718   174.6964            42    -56.0000
29718   184.6964            20    -22.0000
39718   194.6964            10    -10.0000
49703   204.6964             5     -5.0000
59703   214.6964           256    251.0000


In [7]:
## Enriching Dataset

df=df_dict['AC']
param='Ground_Truth'
stats_params = {  
            'IMU': 
                ['GyrX', 'GyrY', 'GyrZ', 'AccX', 'AccY', 'AccZ'],
            # 'RAC':
            #     ['ACCx','ACCy','ACCz'],
            # 'RGY':
            #     ['GYROx','GYROy','GYROz'],
            'ATT':
                ['Roll', 'Pitch', 'Yaw']
        }


# Add attack column into based on timestamps from the transition dataframe
for k in stats_params.keys():
    attack0 = df[param].iloc[0]
    attack = list()
    this_df = df_dict[k]
    tr0 = 0
    for tr in df_transition['timestamp'].values:
        data_size = len(this_df[(this_df['timestamp']>tr0) & (this_df['timestamp']<=tr)])
        # print('adding {} data points to {} points'.format(data_size, len(attack)))
        attack += [attack0]*data_size
        attack0 = df_transition['Ground_Truth'][df_transition['timestamp']==tr].values[0]
        tr0 = tr
    # Compute last segment
    data_size = len(this_df[this_df['timestamp']>tr])
    # print('adding {} data points to {} points'.format(data_size, len(attack)))
    attack += [attack0]*data_size
    this_df['attack'] = attack
    
    # Add 'update' column with the time diference between data points
    this_df['update'] = this_df['timestamp'].diff()

    df_dict[k] = this_df

update_dict = dict()
for ds in datasets[:-1]:
    df_update = pd.DataFrame()
    df_update['timestamp'] = df_dict[ds]['timestamp']
    df_update['update'] = df_dict[ds]['timestamp'].diff()
    update_dict[ds] = df_update



## Statistical Result

stat_dict = dict()
param_set = list()

for k,values in stats_params.items():
    for v in values+['update']:
        this_df = df_dict[k]
        # tr0 = 0
        # compute mean and std deviation for all variables in 'stats_params'
        atk_set = this_df['attack'].unique()
        for atk in atk_set:
            if 'update' in v:
                filtered_list = 1/this_df[v][this_df['attack']==atk]
            else:
                filtered_list = this_df[v][this_df['attack']==atk]

            mean_value = filtered_list.mean()
            std_value = filtered_list.std()

            # fill up the stat_dict dictionary by attack
            name_code = '{}.{}'.format(k, v)
            param_set.append(name_code)
            if atk in stat_dict:
                    stat_dict[atk][name_code] = {'mean': mean_value, 'std':std_value}
            else:
                stat_dict[atk] = {name_code:{'mean': mean_value, 'std':std_value}}

# convert stats into a dataframe
df_columns = ['Freq']
for i in list(set(param_set)):
    df_columns+= [i+'.mean', i+'.std']
stat_df = pd.DataFrame(columns=df_columns)
stat_df['Freq'] = atk_set
for freq in stat_dict:
    for param in stat_dict[freq]:
        stat_df.loc[stat_df['Freq']==freq,param+'.mean'] = stat_dict[freq][param]['mean']
        stat_df.loc[stat_df['Freq']==freq,param+'.std'] = stat_dict[freq][param]['std']


print(stat_df.loc[stat_df['Freq']!=0])
print(stat_df.to_latex())

   Freq IMU.GyrZ.mean IMU.GyrZ.std ATT.Roll.mean ATT.Roll.std IMU.update.mean  \
0   188       -0.0017       0.0005       -5.8281       0.3074        143.0891   
1    98       -0.0015       0.0030       -6.1542       0.2152        125.1028   
2    42       -0.0018       0.0003       -5.7774       0.0396        125.1727   
3    20       -0.0019       0.0012       -6.2938       0.3313        125.1968   
4    10       -0.0020       0.0007       -6.6038       0.0847        125.1283   
5     5       -0.0020       0.0007       -6.3716       0.0418        125.1901   
6   256       -0.0021       0.0010       -5.8192       0.2126        345.5446   

  IMU.update.std ATT.Pitch.mean ATT.Pitch.std ATT.Yaw.mean  ... IMU.AccX.mean  \
0        73.9056        18.6050        0.6241      59.0179  ...        2.9890   
1         6.4646        18.0219        0.1285      70.1297  ...        2.9916   
2         5.6543        17.9971        0.0511     109.4218  ...        3.0003   
3         5.6874        17.

In [8]:
import plot_formatting as pfg

# fig = make_subplots(rows=4, cols=1,
#                     shared_xaxes=True,
#                     vertical_spacing = 0.02,
#                     row_heights=[.25]*2 + [.25]*2)

# colorset = px.colors.qualitative.Vivid
layout = go.Layout( autosize=True, margin={'l': 0, 'r': 0, 't': 0, 'b': 0})
fig = go.Figure(layout=layout)
colorset = px.colors.qualitative.Vivid

# Plot data source
plot_params = {  
            'IMU': 
                ['GyrX', 'AccX', 'update'],
            # 'RAC':
            #     ['ACCx', 'update'],
            # 'RGY':
            #     ['GYROx', 'update'],
            'ATT':
                ['Roll', 'Pitch', 'Yaw', 'update'],
        }

# Legend labels
plot_labels = {'RAC':'Accel', 
               'RGY': 'Gyro',
               'ATT': 'EKF Estimation',
               'IMU': 'IMU'}

x_range = [153.5,157]

color_ix = 0
for k,values in plot_params.items():
    for v in values:
        row_num  = 1
        this_df = df_dict[k]
        this_df = this_df.loc[(this_df['timestamp']>x_range[0]) & (this_df['timestamp']<x_range[1])]
        if 'G' in v:
            row_num = 2

        tr0 = 0
        # if 'ATT' in k and not ('update' in v):
        #     fig.add_trace(go.Scatter(x=this_df['timestamp'], y=this_df[v],
        #             name= v,
        #             mode='lines+markers',
        #             legendgroup=k,
        #             line=dict(color=colorset[color_ix]),
        #             showlegend=True,
        #             legendrank=3,
        #             legendgrouptitle_text='Attitude'),
        #         row=3,col=1)
        # PLotting update frequencies
        if 'update' in v:
            print(this_df)
            fig.add_trace(go.Scatter(x=this_df['timestamp'], y=1/this_df[v],
                    name= plot_labels[k],
                    mode='lines',
                    line=dict(color=colorset[color_ix]),
                    showlegend=True,
                    # legendrank=5,
                    # legendgroup='freq',
                    # legendgrouptitle_text='Frequency',
                    opacity=0.5))
        # else:
        #     fig.add_trace(go.Scatter(x=this_df['timestamp'], y=this_df[v],
        #             name= v,
        #             mode='lines+markers',
        #             legendgroup=v,
        #             line=dict(color=colorset[color_ix]),
        #             showlegend=False),
        #         row=row_num,col=1)


        color_ix += 1

# Plot attacks
df = df.loc[(df['timestamp']>x_range[0]) & (df['timestamp']<x_range[1])]
# inlcude attack transitions as vertical lines in the figure 
for v in df_transition['timestamp'][(df_transition['timestamp']>x_range[0]) & (df_transition['timestamp']<x_range[1])].values:
    fig.add_vline(x=v)

fig.add_vline(x=154.713)

# beautify plots
fig = pfg.beautify_plot(fig)

subplot_titles=['value', 'data rate', 'freq'],

#  Formating x-axis
fig['layout']['xaxis'].update(range=x_range)
# fig['layout']['xaxis1'].update(side='top', mirror=True)
# fig['layout']['xaxis2'].update(side='top', mirror=True)
# fig['layout']['xaxis3'].update(mirror=True)
fig['layout']['xaxis'].update(title='Time [s]', mirror=True)
fig['layout'].update(width=800, height=400, font_size=25)
# fig['layout'].update(yaxis_type="log", yaxis_range=[.0045,2])

# Formating y-axis
# fig['layout']['yaxis1'].update(range=[141.52,141.53], tickvals = [141.52, 141.53],)
# fig['layout']['yaxis1'].update(title='Accelerometer X [m/s/s]')
# # fig['layout']['yaxis2'].update(range=[0.45,0.55])
# # fig['layout']['yaxis3'].update(range=[31.47,31.5], tickvals = [31.47,31.5],)
# fig['layout']['yaxis2'].update(title='Gyro X [rad/s]')
# # fig['layout']['yaxis4'].update(range=[-0.015,0.015])
fig['layout']['yaxis'].update(title='Frequency [Hz]')
# fig['layout']['yaxis3'].update(title='Attitude [°]')
fig['layout']['yaxis'].update(range=[1,3], type='log')

fig['layout']['legend'].update(orientation='h',
                                x=0.55,
                                y=1,
                                   )

fig.show()


     timestamp     TimeUS  I   GyrX   GyrY    GyrZ   AccX   AccY    AccZ  EG  \
1     153.5241  153524089  0 0.0066 0.0103 -0.0029 2.9725 1.0049 -9.1684   0   
2     153.5268  153526819  0 0.0079 0.0126 -0.0031 2.9745 1.0073 -9.1654   0   
3     153.5568  153556844  0 0.0090 0.0104 -0.0027 2.9879 1.0061 -9.1737   0   
4     153.5591  153559082  0 0.0099 0.0096 -0.0041 2.9872 1.0060 -9.1729   0   
5     153.5620  153562037  0 0.0072 0.0075 -0.0035 2.9849 1.0052 -9.1717   0   
..         ...        ... ..    ...    ...     ...    ...    ...     ...  ..   
411   156.9623  156962318  0 0.0064 0.0146 -0.0016 3.0003 0.9999 -9.1701   0   
412   156.9707  156970711  0 0.0064 0.0140 -0.0018 3.0009 0.9992 -9.1694   0   
413   156.9783  156978301  0 0.0065 0.0121 -0.0021 3.0015 0.9988 -9.1683   0   
414   156.9864  156986426  0 0.0066 0.0093 -0.0020 3.0021 0.9985 -9.1670   0   
415   156.9949  156994865  0 0.0065 0.0063 -0.0015 3.0027 0.9985 -9.1654   0   

     EA       T  GH  AH   GHz  AHz  att

In [9]:
fig.write_image('figs/MPU600Frequency.pdf')